In [9]:
import pandas as pd
import numpy as np
import json

tabela = pd.read_csv("PATRIMONIO 12 2023.csv", encoding='windows-1252',delimiter=';')

display(tabela)

,Nº inventário,Localização,Denominação,Incorporação em
0,1221588.0,21080001,SENAI Ricardo Lerner,02/12/2009
1,1221668.0,21080001,SENAI Ricardo Lerner,01/12/2016
2,724781.0,21080013,PROJETOR MULTIMIDIA 1200 ANSI LUMENS,31/07/2004
3,724782.0,21080013,PROJETOR MULTIMIDIA 1200 ANSI LUMENS,31/07/2004
4,724783.0,21080013,PROJETOR MULTIMIDIA 1200 ANSI LUMENS,31/07/2004
...,...,...,...,...
4819,1278680.0,21080050,PÚLPITO ACRÍLICO 120CM,31/07/2023
4820,1279970.0,21080003,DETECTOR DE 5 GASES C/KIT DE CALIBRAÇÃO,19/09/2023
4821,1281779.0,21080059,ROBÔ COLABORATIVO 5 KG,14/11/2023
4822,1281780.0,21080059,ROBÔ COLABORATIVO 5 KG,14/11/2023


In [10]:
display(tabela.info())

tabela['Nº inventário'].fillna(0, inplace=True)

tabela['Nº inventário'] = tabela['Nº inventário'].astype(int)

display(tabela.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4824 entries, 0 to 4823
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Nº inventário    4773 non-null   float64
 1   Localização      4824 non-null   int64  
 2   Denominação      4824 non-null   object 
 3   Incorporação em  4824 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 150.9+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4824 entries, 0 to 4823
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Nº inventário    4824 non-null   int64 
 1   Localização      4824 non-null   int64 
 2   Denominação      4824 non-null   object
 3   Incorporação em  4824 non-null   object
dtypes: int64(2), object(2)
memory usage: 150.9+ KB


C:\Users\DELL\AppData\Local\Temp\ipykernel_14960\3676236845.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  tabela['Nº inventário'].fillna(0, inplace=True)


None

In [11]:
tabela['Sala'] = np.where(tabela['Localização'] == 21080001, 'SALA 01', np.where(tabela['Localização'] == 21080013, 'SALA 13','OUTRA'))
display(tabela)

display(tabela.info())

,Nº inventário,Localização,Denominação,Incorporação em,Sala
0,1221588,21080001,SENAI Ricardo Lerner,02/12/2009,SALA 01
1,1221668,21080001,SENAI Ricardo Lerner,01/12/2016,SALA 01
2,724781,21080013,PROJETOR MULTIMIDIA 1200 ANSI LUMENS,31/07/2004,SALA 13
3,724782,21080013,PROJETOR MULTIMIDIA 1200 ANSI LUMENS,31/07/2004,SALA 13
4,724783,21080013,PROJETOR MULTIMIDIA 1200 ANSI LUMENS,31/07/2004,SALA 13
...,...,...,...,...,...
4819,1278680,21080050,PÚLPITO ACRÍLICO 120CM,31/07/2023,OUTRA
4820,1279970,21080003,DETECTOR DE 5 GASES C/KIT DE CALIBRAÇÃO,19/09/2023,OUTRA
4821,1281779,21080059,ROBÔ COLABORATIVO 5 KG,14/11/2023,OUTRA
4822,1281780,21080059,ROBÔ COLABORATIVO 5 KG,14/11/2023,OUTRA


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4824 entries, 0 to 4823
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Nº inventário    4824 non-null   int64 
 1   Localização      4824 non-null   int64 
 2   Denominação      4824 non-null   object
 3   Incorporação em  4824 non-null   object
 4   Sala             4824 non-null   object
dtypes: int64(2), object(3)
memory usage: 188.6+ KB


None

In [12]:
salasEspecificas = [21080001, 21080013]
tabelaFiltrada = tabela[tabela['Localização'].isin(salasEspecificas)]

contagemItens = tabelaFiltrada.groupby('Localização')['Nº inventário'].count().reset_index()
display(contagemItens)


,Localização,Nº inventário
0,21080001,6
1,21080013,162


In [30]:
grupoSalas = tabela[tabela['Localização'].isin(salasEspecificas)].groupby('Sala').agg({
    'Nº inventário': 'count',
    'Localização': lambda x: list(set(x)),  
    'Denominação': lambda x: x.tolist()  # Converter a coluna 'Nº inventário' em uma lista
}).reset_index()
resultado = {}
for _, row in grupoSalas.iterrows():
    resultado[row['Sala']] = {
        'quantidade de Nº inventário': row['Nº inventário'],
        'localização': row['Localização'],
        'Denominação': row['Denominação']
    }

resultado_json = json.dumps(resultado, indent=2, ensure_ascii=False)

print(resultado_json)


{
  "SALA 01": {
    "quantidade de Nº inventário": 6,
    "localização": [
      21080001
    ],
    "Denominação": [
      "SENAI Ricardo Lerner",
      "SENAI Ricardo Lerner",
      "MESA COM TAMPO EM GOTA MALIBU - DC 1692",
      "MESA COM TAMPO EM GOTA MALIBU - DC 1692",
      "MESA COM TAMPO EM GOTA MALIBU - DC 1692",
      "MESA EM L COM DIVISÓRIA MALIBU - DC 1693"
    ]
  },
  "SALA 13": {
    "quantidade de Nº inventário": 162,
    "localização": [
      21080013
    ],
    "Denominação": [
      "PROJETOR MULTIMIDIA 1200 ANSI LUMENS",
      "PROJETOR MULTIMIDIA 1200 ANSI LUMENS",
      "PROJETOR MULTIMIDIA 1200 ANSI LUMENS",
      "PROJETOR MULTIMIDIA 1200 ANSI LUMENS",
      "PROJETOR MULTIMIDIA 1200 ANSI LUMENS",
      "CAIXA ACUSTICA TIPO ARANDELA 40W",
      "CAIXA ACUSTICA TIPO ARANDELA 40W",
      "CAIXA ACUSTICA TIPO ARANDELA 40W",
      "CAIXA ACUSTICA TIPO ARANDELA 40W",
      "CAIXA ACUSTICA TIPO ARANDELA 40W",
      "CAIXA ACUSTICA TIPO ARANDELA 40W",
      "CAIXA 